In [2]:
import setuptools
import os
import re
import string
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

import numpy as np
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import scipy.sparse

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

In [12]:
CONFIG = {
    "data_path": "data.csv",
    "test_size": 0.2,
    "mlflow_tracking_uri": "https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow",
    "dagshub_repo_owner": "SoumilMalik24",
    "dagshub_repo_name": "Sentiment-Analysis-with-MLOps",
    "experiment_name": "Bow vs TfIdf"
}

In [4]:
mlflow.set_tracking_uri(CONFIG["mlflow_tracking_uri"])
dagshub.init(repo_owner=CONFIG["dagshub_repo_owner"], repo_name=CONFIG["dagshub_repo_name"], mlflow=True)
mlflow.set_experiment(CONFIG["experiment_name"])

Accessing as SoumilMalik24

Initialized MLflow to track repo "SoumilMalik24/Sentiment-Analysis-with-MLOps"

Repository SoumilMalik24/Sentiment-Analysis-with-MLOps initialized!

2026/01/22 10:02:06 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/5577bd40c8eb4b298fb0117261a44f59', creation_time=1769056326893, experiment_id='1', last_update_time=1769056326893, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [5]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return text.lower()

def removing_punctuations(text):
    return re.sub(f"[{re.escape(string.punctuation)}]", ' ', text)

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def normalize_text(df):
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [6]:
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        df = normalize_text(df)
        df = df[df['sentiment'].isin(['positive', 'negative'])]
        df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1}).infer_objects(copy=False)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise

In [7]:
VECTORIZERS = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

ALGORITHMS = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [8]:
def train_and_evaluate(df):
    with mlflow.start_run(run_name="All Experiments") as parent_run:
        for algo_name, algorithm in ALGORITHMS.items():
            for vec_name, vectorizer in VECTORIZERS.items():
                with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                    try:
                        # Feature extraction
                        X = vectorizer.fit_transform(df['review'])
                        y = df['sentiment']
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=CONFIG["test_size"], random_state=42)

                        # Log preprocessing parameters
                        mlflow.log_params({
                            "vectorizer": vec_name,
                            "algorithm": algo_name,
                            "test_size": CONFIG["test_size"]
                        })

                        # Train model
                        model = algorithm
                        model.fit(X_train, y_train)

                        # Log model parameters
                        log_model_params(algo_name, model)

                        # Evaluate model
                        y_pred = model.predict(X_test)
                        metrics = {
                            "accuracy": accuracy_score(y_test, y_pred),
                            "precision": precision_score(y_test, y_pred),
                            "recall": recall_score(y_test, y_pred),
                            "f1_score": f1_score(y_test, y_pred)
                        }
                        mlflow.log_metrics(metrics)

                        # Log model
                        # mlflow.sklearn.log_model(model, "model")
                        input_example = X_test[:5] if not scipy.sparse.issparse(X_test) else X_test[:5].toarray()
                        mlflow.sklearn.log_model(model, "model", input_example=input_example)

                        # Print results for verification
                        print(f"\nAlgorithm: {algo_name}, Vectorizer: {vec_name}")
                        print(f"Metrics: {metrics}")

                    except Exception as e:
                        print(f"Error in training {algo_name} with {vec_name}: {e}")
                        mlflow.log_param("error", str(e))

In [9]:
def log_model_params(algo_name, model):
    """Logs hyperparameters of the trained model to MLflow."""
    params_to_log = {}
    if algo_name == 'LogisticRegression':
        params_to_log["C"] = model.C
    elif algo_name == 'MultinomialNB':
        params_to_log["alpha"] = model.alpha
    elif algo_name == 'XGBoost':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
    elif algo_name == 'RandomForest':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["max_depth"] = model.max_depth
    elif algo_name == 'GradientBoosting':
        params_to_log["n_estimators"] = model.n_estimators
        params_to_log["learning_rate"] = model.learning_rate
        params_to_log["max_depth"] = model.max_depth

    mlflow.log_params(params_to_log)

In [13]:
df = load_data(CONFIG["data_path"])

train_and_evaluate(df)

2026/01/22 10:06:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:06:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/01/22 10:06:40 INFO mlflow.models.model: Found the following environment variables used during model inference: [NEWSAPI_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.



Algorithm: LogisticRegression, Vectorizer: BoW
Metrics: {'accuracy': 0.8, 'precision': 0.75, 'recall': 0.8478260869565217, 'f1_score': 0.7959183673469388}
🏃 View run LogisticRegression with BoW at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/bfc8d8a11811425d84a34c5e5f8c3fec
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1


2026/01/22 10:06:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:07:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Algorithm: LogisticRegression, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.78, 'precision': 0.8157894736842105, 'recall': 0.6739130434782609, 'f1_score': 0.7380952380952381}
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/6fcaf1154fc0481a9dc0068d4aad152c
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1


2026/01/22 10:07:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:07:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


...


Algorithm: RandomForest, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.8, 'precision': 0.782608695652174, 'recall': 0.782608695652174, 'f1_score': 0.782608695652174}
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/bd29064d3e874a8d8c8205eeb83938c2
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1


2026/01/22 10:09:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:09:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Algorithm: GradientBoosting, Vectorizer: BoW
Metrics: {'accuracy': 0.68, 'precision': 0.6521739130434783, 'recall': 0.6521739130434783, 'f1_score': 0.6521739130434783}
🏃 View run GradientBoosting with BoW at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/42e538d965a047e6994a2c9ebcf18cf6
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1


2026/01/22 10:10:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/22 10:10:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Algorithm: GradientBoosting, Vectorizer: TF-IDF
Metrics: {'accuracy': 0.73, 'precision': 0.6862745098039216, 'recall': 0.7608695652173914, 'f1_score': 0.7216494845360825}
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/9aed5a4eb20e464fa9de4b326cd96d5f
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1
🏃 View run All Experiments at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1/runs/9ebb758b05e54c88af2173b4f960f13f
🧪 View experiment at: https://dagshub.com/SoumilMalik24/Sentiment-Analysis-with-MLOps.mlflow/#/experiments/1
